In [1]:
import os
import json

from typing import List, Dict, Any, Tuple

from formating import format_date, format_value, escape_string
from crud import insert_records, delete_records, fetch_records

In [2]:
def convert_json_to_sql(json_data: Dict[str, Any], column_mapping: List[Dict[str, str]], target_tablename:str) -> str:
    """Convert JSON data to SQL insert statement with multiple VALUES."""
    
    # Get column names
    columns = [mapping['target'] for mapping in column_mapping]
    columns_sql = ', '.join(columns)
    
    # Collect all value sets
    all_values = []
    for record in json_data:
        values = []
        for mapping in column_mapping:
            source_field = mapping['source']
            value_type = mapping['type']
            try:
                value = mapping['default']
            except KeyError:
                value = record.get(source_field)    
            formatted_value = format_value(value, value_type)
            values.append(formatted_value)
        all_values.append(f"({', '.join(str(value) for value in values)})")
    
    # Create single INSERT statement with multiple VALUES
    values_sql = ',\n'.join(all_values)
    sql_statement = f"INSERT INTO {target_tablename} ({columns_sql})\nVALUES\n{values_sql};"
    
    return sql_statement

In [3]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
assets_dir = os.path.join(parent_dir, '_assets')
db_dir = os.path.join(parent_dir, 'fetusapp')
db_filename = 'data.sqlite'
data_filename = 'detoraki_fetusdata.json'
mapping_filename = 'column_mapping.json'

db_path = os.path.join(db_dir, db_filename)
json_file = os.path.join(assets_dir, data_filename)
mapping_file = os.path.join(current_dir, mapping_filename)

Load the source data

In [15]:
with open(json_file, 'r', encoding='utf-8') as f:
        json_data = json.load(f)

Load the column mapping from the JSON file

In [16]:
with open(mapping_file, 'r', encoding='utf-8') as f:
        column_mapping = json.load(f)

### Migrate Patients

Generate the sql output and save it

In [34]:
source_tablename = 's_org_ext'
target_tablename = 'patients'
data_index = 3

sql_output = convert_json_to_sql(json_data[data_index]['data'], column_mapping[source_tablename], target_tablename = target_tablename)
# print(sql_output)
output_file = os.path.join(assets_dir, f'{target_tablename}.sql')
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(sql_output)

In [28]:
patient_data = json_data[data_index]['data']
null_lastname_ids = [item['s_org_extID'] for item in patient_data if item['lastName'] == ""]
print(null_lastname_ids)

['96', '245', '507', '513', '521', '758', '814', '1277', '1417']


In [30]:
delete_records(target_tablename, db_path)

In [32]:
insert_records(target_tablename, db_path)

In [33]:
fetch_records(target_tablename, db_path)

(3, 2, 2, None, '2022-12-19 22:01:12', 'Φανή', 'Βασιλειάδου', 'Σπυρίδωνας', '1945-03-04', 'single', 'Ελληνική', 'Συνταξιούχος ΟΤΕ', 'Λέσβου', '112', 'Αθήνα', '17237', 'Αττικής', '2109731000', '6977221781', '2299073763', 'zdetor54@gmail.com', 'other', 'Axxa Ασφαλιστική', '13745541', 'Γεώργιος Δετοράκης', '1940-08-08', 'Συνταξιούχος ΟΤΕ', 0)
(11, 2, 2, None, '2013-09-22 02:35:13', 'Μαρία', 'Παναγοπούλου', 'Κώστας', None, None, None, None, None, '0', None, '0', None, None, None, None, None, None, None, None, None, None, None, 0)
(41, 2, 2, '2013-09-10 22:15:33', '2013-09-10 22:15:34', 'Μαρία', 'Κωνσταντινίδου', None, None, None, None, None, None, None, None, '0', None, None, None, None, None, None, None, None, None, None, None, 1)
(48, 2, 2, '2013-09-15 15:18:53', '2013-09-15 22:02:56', 'Μαρία', 'Ρεπούση', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 1)
(54, 2, 2, None, '2013-10-24 23:51:03', 'Μαρία', 'Κοσμίδη', No

### Patients migrated